In [ ]:
# default_exp pretrained

In [ ]:
#all_slow

# Pretrained

> fast.ai ULMFiT helpers to easily use pretrained models

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import json
from fastai.text.all import SentencePieceTokenizer, language_model_learner, \
                            text_classifier_learner, untar_data, Path, patch, \
                            LMLearner, os, pickle, shutil, AWD_LSTM, accuracy, \
                            Perplexity, delegates

In [ ]:
#export
def _get_config(path):
    with open(path/'model.json', 'r') as f:
        config = json.load(f)
    return config

In [ ]:
#export
def _get_pretrained_model(url):
    fname = f"{url.split('/')[-1]}.tgz"
    path = untar_data(url, fname=fname, c_key='model')
    return path

In [ ]:
#export
def _get_direction(backwards):
    return 'bwd' if backwards else 'fwd'

In [ ]:
#hide
assert(_get_direction(backwards=False) == 'fwd')
assert(_get_direction(backwards=True) == 'bwd')

Get `model` and `vocab` files from path. 

In [ ]:
#export
def _get_model_files(path, backwards=False):
    direction = _get_direction(backwards)
    config = _get_config(path/direction)
    try: 
        model_path = path/direction
        model_file = list(model_path.glob(f'*model.pth'))[0]
        vocab_file = list(model_path.glob(f'*vocab.pkl'))[0]
        fnames = [model_file.absolute(),vocab_file.absolute()]
    except IndexError: print(f'The model in {model_path} is incomplete, download again'); raise
    fnames = [str(f.parent/f.stem) for f in fnames]
    return fnames

## Tokenizer

Get `tokenizer` from model-config. Tokenizer parameters in `model.json` will be passed to the Tokenizer. As of now SentencePieceTokenizer is hard-coded.

In [ ]:
#export
def tokenizer_from_pretrained(url, backwards=False, **kwargs):
    path = _get_pretrained_model(url)
    direction = _get_direction(backwards)
    config = _get_config(path/direction)
    tok = None
    if config['tokenizer']['class'] == 'SentencePieceTokenizer':
        tok = SentencePieceTokenizer(**config['tokenizer']['params'], **kwargs)
    return tok

## Language Model Learner

Create `langauge_model_learner` from pretrained model-URL. 

In [ ]:
#export
@delegates(language_model_learner)
def language_model_from_pretrained(dls, url=None, backwards=False, metrics=None, **kwargs):
    arch = AWD_LSTM # TODO: Read from config
    path = _get_pretrained_model(url)
    fnames = _get_model_files(path)
    metrics = [accuracy, Perplexity()] if metrics == None else metrics
    return language_model_learner(dls, 
                                  arch, 
                                  pretrained=True, 
                                  pretrained_fnames=fnames, 
                                  metrics=metrics,
                                  **kwargs)

Saves a trained or fine-tuned language model. 

In [ ]:
#export
def _get_model_path(learn=None, path=None):
    path = (learn.path/learn.model_dir) if not path else Path(path)
    if not path.exists(): os.makedirs(path, exist_ok=True)
    return path

Saves the following model files to `path`:
- Model (`lm_model.pth`)
- Encoder (`lm_encoder.pth`)
- Vocab from dataloaders (`lm_vocab.pkl`)
- SentencePieceModel (`spm/`)

In [ ]:
#export
@patch
def save_lm(x:LMLearner, path=None, with_encoder=True):
    path = _get_model_path(x, path)
    x.to_fp32()
    # save model
    x.save((path/'lm_model').absolute(), with_opt=False)
    
    # save encoder
    if with_encoder:
        x.save_encoder((path/'lm_encoder').absolute())

    # save vocab
    with open((path/'lm_vocab.pkl').absolute(), 'wb') as f:
        pickle.dump(x.dls.vocab, f)
        
    # copy SPM if path not spm path
    spm_path = Path(x.dls.tok.cache_dir)
    if path.absolute() != spm_path.absolute():
        target_path = path/'spm'
        if not target_path.exists(): os.makedirs(target_path, exist_ok=True)
        shutil.copyfile(spm_path/'spm.model', target_path/'spm.model')
        shutil.copyfile(spm_path/'spm.vocab', target_path/'spm.vocab')
    
    return path

## Text Classifier

In [ ]:
#def vocab_from_lm(learn=None, path=None):
#    path = _get_model_path(learn, path)
#    with open((path/'lm_vocab.pkl').absolute(), 'rb') as f:
#        return pickle.load(f)

In [ ]:
#def spm_from_lm(learn=None, path=None):
#    path = _get_model_path(learn, path)

Create `text_classifier_learner` from fine-tuned model path (saved with `learn.save_lm()`).

In [ ]:
#export
@delegates(text_classifier_learner)
def text_classifier_from_lm(dls, path=None, backwards=False, **kwargs):
    arch = AWD_LSTM # TODO: Read from config
    path = _get_model_path(path=path)
    learn = text_classifier_learner(dls, arch, pretrained=False, **kwargs)
    learn.load_encoder((path/'lm_encoder').absolute())
    return learn

# Tests - Tokenizer, LM and Classifier

In [ ]:
#hide
url = 'http://localhost:8080/ulmfit-dewiki'
tok = tokenizer_from_pretrained(url)
assert(tok.vocab_sz == 15000)

In [ ]:
#hide
from fastai.text.all import AWD_LSTM, DataBlock, TextBlock, ColReader, RandomSplitter
import pandas as pd

backwards = False

df = pd.read_csv(Path('_test/data_lm_sample.csv'))

dblocks = DataBlock(blocks=(TextBlock.from_df('text', tok=tok, is_lm=True, backwards=backwards)),
                    get_x=ColReader('text'), 
                    splitter=RandomSplitter(valid_pct=0.1, seed=42))
dls = dblocks.dataloaders(df, bs=128)

learn = language_model_from_pretrained(dls, url=url, backwards=backwards)
learn.fit_one_cycle(1)

/home/florian/miniconda3/envs/fastai/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,6.456911,6.521295,0.169497,679.457458,00:00


In [ ]:
#hide
path = learn.save_lm()
vocab = learn.dls.vocab

In [ ]:
#hide
from fastai.text.all import AWD_LSTM, DataBlock, TextBlock, ColReader, RandomSplitter, CategoryBlock
import pandas as pd

backwards = False

df = pd.read_csv(Path('_test/data_class_sample.csv'))

dblocks = DataBlock(blocks=(TextBlock.from_df('text', tok=tok, vocab=vocab, backwards=backwards), CategoryBlock),
                    get_x=ColReader('text'), 
                    get_y=ColReader('label'))
dls = dblocks.dataloaders(df, bs=128)

learn = text_classifier_from_lm(dls, path=path, backwards=backwards)
learn.fit_one_cycle(1)
learn.get_preds()

/home/florian/miniconda3/envs/fastai/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


epoch,train_loss,valid_loss,time
0,0.746191,0.686059,00:01


(tensor([[0.5208, 0.4792],
         [0.4542, 0.5458],
         [0.5424, 0.4576],
         [0.5126, 0.4874],
         [0.5279, 0.4721],
         [0.5018, 0.4982],
         [0.5503, 0.4497],
         [0.5119, 0.4881],
         [0.5376, 0.4624],
         [0.5206, 0.4794],
         [0.5527, 0.4473],
         [0.4942, 0.5058],
         [0.4967, 0.5033],
         [0.5023, 0.4977],
         [0.4966, 0.5034],
         [0.5292, 0.4708],
         [0.4937, 0.5063],
         [0.5089, 0.4911],
         [0.4843, 0.5157],
         [0.5148, 0.4852],
         [0.4945, 0.5055],
         [0.5097, 0.4903],
         [0.4925, 0.5075],
         [0.5141, 0.4859],
         [0.5034, 0.4966],
         [0.5540, 0.4460],
         [0.5136, 0.4864],
         [0.5497, 0.4503],
         [0.5158, 0.4842],
         [0.4923, 0.5077],
         [0.5600, 0.4400],
         [0.4952, 0.5048],
         [0.5106, 0.4894],
         [0.4843, 0.5157],
         [0.5156, 0.4844],
         [0.5331, 0.4669],
         [0.5203, 0.4797],
 